In [ ]:
import spacy
import random
import pandas as pd
import json
from urllib.request import urljoin 
from bs4 import BeautifulSoup 
import requests 
from urllib.request import urlparse 
from urllib.error import HTTPError
from urllib.error import URLError

In [ ]:
from google.colab import drive
drive.mount("my-drive")

Mounted at my-drive


In [ ]:
nlp_model=spacy.load('my-drive/My Drive/nlp_model')

In [ ]:
def selected_tags(tag):
  return tag!='a' and tag!='style' and tag!='script' and tag!='title'
def extract_info(url):
  try:
    page=requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    content=soup.find_all(selected_tags)
    all_links=soup.find_all('a')
    content=content.strip()
    email=set()
    personal_website=set()
    for link in all_links:
      if '@' in link.text:
        email.add(link.text.strip())
      elif 'email' in str(link.text).lower():
        email.add(link.get('href'))
      if 'website' in str(link.text).lower():
        personal_website.add(link.get('href'))
      elif 'profile' in str(link.text).lower():
        personal_website.add(link.get('href'))
    return content,list(email),list(personal_website)
  except HTTPError as e:
    print("404 Not Found")
    return "",[],[]
  except URLError as e:
    print("The server can't be reached")
    return "",[],[]
  except:
    print("Some unexpected error occurred")
    return "",[],[]

In [ ]:
def update_info(path,df,content_df):
  ent_weights={'Specialization':10,'Address':30,'Experience':20,'Email':50,'MobileNumber':40,'Qualification':20,'Designation':20,'PersonalWebsite':25,'Department':20,'Degree':20}
  for i in range(len(df)):
    total_weight=0
    Specialization=''
    Address=''
    Experience=''
    MobileNumber=''
    Qualification=''
    Designation=''
    Department=''
    Degree=''
    print('Predicting')
    doc=nlp_model(str(content_df.at[i,'content']))
    for ent in doc.ents:
      if len(ent.text)<3:
        continue
      print(ent.text+'\t'+ent.label_)
      if ent.label_=='Specialization':
        if ent.text.lower() not in str(df.at[i,'Area of Specialization(in words)']).lower() and ent.text.lower() not in Specialization.lower():
          Specialization=Specialization+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Address':
        if ent.text.lower() not in str(df.at[i,'Corresponding Address']).lower() and ent.text.lower() not in Address.lower():
          Address=Address+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Experience':
        if ent.text.lower() not in str(df.at[i,'Experience']).lower() and ent.text.lower() not in Experience.lower():
          Experience=Experience+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='MobileNumber':
        if ent.text.lower() not in str(df.at[i,'Mobile No']).lower() and ent.text.lower() not in MobileNumber.lower():
          MobileNumber=MobileNumber+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Qualification':
        if ent.text.lower() not in str(df.at[i,'Qualification']).lower() and ent.text.lower() not in Qualification.lower():
          Qualification=Qualification+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Designation':
        if ent.text.lower() not in str(df.at[i,'Designation']).lower() and ent.text.lower() not in Designation.lower():
          Designation=Designation+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Department':
        if ent.text.lower() not in str(df.at[i,'Department']).lower() and ent.text.lower() not in Department.lower():
          Department=Department+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Degree':
        if ent.text.lower() not in str(df.at[i,'Doctorate Degree']).lower() and ent.text.lower() not in Degree.lower():
          Degree=Degree+', '+ent.text
          total_weight+=ent_weights[ent.label_]
    if content_df.at[i,'email'] not in df.at[i,'E-mail']:
      total_weight+=ent_weights['Email']
    if content_df.at[i,'personal_website'] not in df.at[i,'Personal Website']:
      total_weight+=ent_weights['PersonalWebsite'] 
    if total_weight>50:
      if len(Specialization)!=0:
        if 'not found' in df.at[i,'Area of Specialization(in words)'].lower():
          df.at[i,'Area of Specialization(in words)']=Specialization
        else:
          df.at[i,'Area of Specialization(in words)']=df.at[i,'Area of Specialization(in words)']+Specialization
      if len(Address)!=0:
        if 'not found' in df.at[i,'Corresponding Address'].lower():
          df.at[i,'Corresponding Address']=Address
        else:
          df.at[i,'Corresponding Address']=df.at[i,'Corresponding Address']+Address
      if len(Experience)!=0:
        if 'not found' in df.at[i,'Experience'].lower():
          df.at[i,'Experience']=Experience
        else:
          df.at[i,'Experience']=df.at[i,'Experience']+Experience
      if len(MobileNumber)!=0:
        if 'not found' in df.at[i,'Mobile No'].lower():
          df.at[i,'Mobile No']=MobileNumber
        else:
          df.at[i,'Mobile No']=df.at[i,'Mobile No']+MobileNumber
      if len(Qualification)!=0:
        if 'not found' in df.at[i,'Qualification'].lower():
          df.at[i,'Qualification']=Qualification
        else:
          df.at[i,'Qualification']=df.at[i,'Qualification']+Qualification
      if len(Designation)!=0:
        if 'not found' in df.at[i,'Designation'].lower():
          df.at[i,'Designation']=Designation
        else:
          df.at[i,'Designation']=df.at[i,'Designation']+Designation
      if len(Department)!=0:
        if 'not found' in df.at[i,'Department'].lower():
          df.at[i,'Department']=Department
        else:
          df.at[i,'Department']=df.at[i,'Department']+Department
      if len(Degree)!=0:
        if 'not found' in df.at[i,'Doctorate Degree'].lower():
          df.at[i,'Doctorate Degree']=Degree
        else:
          df.at[i,'Doctorate Degree']=df.at[i,'Doctorate Degree']+Degree
      if content_df.at[i,'email'] not in df.at[i,'E-mail']:
        df.at[i,'E-mail']=content_df.at[i,'email']
      if content_df.at[i,'personal_website'] not in df.at[i,'Personal Website']:
        if 'not found' in df.at[i,'Personal Website'].lower():
          df.at[i,'Personal Website']=content_df.at[i,'personal_website']
        else:
          df.at[i,'Personal Website']=df.at[i,'Personal Website']+', '+content_df.at[i,'personal_website']
    

In [ ]:
def update_changes(path,df):
  urls=list(df['URL'])
  info_dict={'content':[],'email':[],'personal_website':[]}
  for i in range(len(urls)):
    url_list=set(urls[i].split(','))
    url_list=list(url_list)
    content=''
    email=[]
    personal_website=[]
    for url in url_list:
      if '.edu' not in url:
        continue
      url=url.strip()
      content1,email1,personal_website1=extract_info(url)      
      content=content+', '+content1
      email.extend(email1)
      personal_website.extend(personal_website1)
    email_str=', '.join([str(everymail) for everymail in email])
    personal_website_str=', '.join([str(everywebsite) for everywebsite in personal_website])
    info_dict['content'].append(content)
    info_dict['email'].append(email_str)
    info_dict['personal_website'].append(personal_website_str)
  content_df=pd.DataFrame.from_dict(info_dict)
  print(len(df))
  print(len(content_df))
  if len(df)==len(content_df):
    update_info(path,df,content_df)

In [ ]:
def read_file(path,option):
  df=''
  if path.endswith('.csv'):
    df=pd.read_csv(path)
  elif path.endswith('.xlsx'):
    df=pd.read_excel(path)
  else:
    print('This file format is not accepted')
    return
  if option==1:
    update_changes(path,df)
  elif option==2:
    find_new_faculty(path,df)
  elif option==3:
    update_changes(path,df)
    find_new_faculty(path,df)